# Completing information about medicines in pharmacy

---

## Imports

#### Standard library imports

In [1]:
import sys

#### Third party imports

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import difflib

import matplotlib.pyplot as plt

#### Local application imports

In [3]:
sys.path.append("../")

In [4]:
%load_ext autoreload
%autoreload 2

from utils.catalog_completion_params import (
    pres_types,
    mistakes_id,
    lev_low_tr,
    lev_high_tr
)

from utils.catalog_completion_funcs import (
    rel_leveshtein,
    fill_meds_info,
    fill_meds_adm
)

---

## Data

### Data to complete

#### Reading data

In [ ]:
df_obj = pd.read_csv("../data/cat_plat_pc.csv")

#### Cleaning data

In [ ]:
## Converting everything to uppercase
df_obj["medicamento"] = df_obj["medicamento"].str.upper()

## Cleaning whitespace
df_obj["medicamento"] = df_obj["medicamento"].str.strip()

# ## New columns to store results
# df_obj["med_ref_match"] = "_"
# df_obj["med_ref_score"] = 0

### Reference data (fanasa)

#### Reading data

In [ ]:
df_ref = pd.read_csv("../data/cat_fansa.csv")

#### Cleaning data

In [ ]:
## Splitting entries based on pattern
# col_splt = df_ref["descripcion"].str.split(" - ", 1, expand=True)
# df_ref["med_ref"] = col_splt[0]
# df_ref["pres_red"] = col_splt[1]
# df_ref.drop(["descripcion"], axis=1, inplace = True)

## Cleaning whitespace
df_ref["descripcion"] = df_ref["descripcion"].str.strip()

### Reference data (SAT)

#### Reading data

In [24]:
df_rsat = pd.read_excel("../data/meds_SAT.xlsx")

#### Cleaning data

In [25]:
## Selecting relevant columns
rc = [
    "CLAVE O CODIGO",
    "SUBCLAVE O CODIGO",
    "NOMBRE GENERICO",
    "NOMBRE DE GRUPO TERAPEUTICO",
    "PRINCIPAL INDICACION",
    "DEMAS INDICACIONES",
]
df_rsat = df_rsat.loc[:, rc].copy()

## Eliminate dupliacted columns
df_rsat.drop_duplicates(subset=["NOMBRE GENERICO", "NOMBRE DE GRUPO TERAPEUTICO"], inplace=True)

## Restart index
df_rsat.reset_index(inplace=True, drop=True)

In [26]:
df_rsat

,CLAVE O CODIGO,SUBCLAVE O CODIGO,NOMBRE GENERICO,NOMBRE DE GRUPO TERAPEUTICO,PRINCIPAL INDICACION,DEMAS INDICACIONES
0,010.000.0101.00,010.000.0101.00,ACIDO ACETILSALICILICO,ANALGESIA,1. ARTRITIS REUMATOIDE,2. OSTEOARTRITIS 3. ESPONDILITIS ANQUILOSANTE ...
1,010.000.5940.00,010.000.5940.00,IBUPROFENO,ANALGESIA,1. DOLOR DE LEVE A MODERADO.,2. FIEBRE.
2,010.000.0108.00,010.000.0108.00,METAMIZOL SODICO,ANALGESIA,1. FIEBRE,2. DOLOR AGUDO O CRONICO 3. ALGUNOS CASOS DE D...
3,010.000.0104.00,010.000.0104.00,PARACETAMOL,ANALGESIA,1. FIEBRE,2. DOLOR AGUDO O CRONICO
4,040.000.2097.00,040.000.2097.00,BUPRENORFINA,ANALGESIA,DOLOR CRONICO DE INTENSIDAD MODERADA A SEVERA ...,NaN
...,...,...,...,...,...,...
1042,FCBCISS.000010,FCBCISS.00010,"GLUCOSAMINA, CONDROITINA",NO DISPONIBLE,NaN,NaN
1043,FCBCISS.00011,FCBCISS.00011,"PINAVERIO, SIMETICONA",NO DISPONIBLE,NaN,NaN
1044,FCBCISS.00012,FCBCISS.00012,INMUNOGLOBULINA G NO MODIFICADA,NO DISPONIBLE,NaN,NaN
1045,FCBCISS.00013,FCBCISS.00013,ISOCONAZOL,NO DISPONIBLE,NaN,NaN


In [27]:
## Saving final result as excel
df_rsat.to_excel("../results/catalogo_sat_rob.xlsx")

---

## Matching data

In [ ]:
df_obj

In [ ]:
df_ref

### Using fuzzy matching
- Notes: 
    1. It takes too long.

In [ ]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [ ]:
dfx = fuzzy_merge(df_obj, df_ref, 'medicamento', 'descripcion', threshold=65)

In [ ]:
df_obj = dfx.copy()

### Using difflib

#### Matching aproximates

#### Loading saved results

In [ ]:
df_obj = pd.read_pickle("../data/match25.pkl")

#### Calculating levenshtein distances of matches

In [ ]:
df_obj["lev_score"] = df_obj.apply(lambda x: rel_leveshtein(x["medicamento"], x["best_match"]), axis=1)

#### Marking all entries that were identified as mistakes

In [ ]:
df_obj["idm"] = 0
"""
Codes:
    a)  0 -> not evaluated
    b)  1 -> confirmed correct
    c) -1 -> confirmed incorrect
"""

In [ ]:
## Results with leveshteing similarity below or above specific treshold
df_obj.loc[df_obj["lev_score"] <= lev_low_tr, ["idm"]] = -1
df_obj.loc[df_obj["lev_score"] >= lev_high_tr, ["idm"]] = 1

## Manual entered mistakes
for mid in mistakes_id:
    df_obj.loc[df_obj.index.isin(mistakes_id[mid]), "idm"] = -1

In [ ]:
df_obj["idm"].value_counts()

---

## Filling table with required info

#### Preparing dataframe

In [ ]:
df_obj["best_match_lst"] = df_obj["best_match"].str.split(" - ")
df_obj["presents"] = df_obj["best_match_lst"].apply(lambda x: x[1] if len(x) > 1 else "check").str.strip()
df_obj["pres_type"] = df_obj["presents"].apply(lambda x: x[:3])
df_obj["pres_dims"] = df_obj["presents"].apply(lambda x: x[3:]).str.strip().str.replace(" ", "")
df_obj["pres_dims_gen"] = df_obj["pres_dims"].str.replace("\d+", "#")

## Dropping unused columns
df_obj.drop(["clave", "match", "match_ct", "best_match_lst"], axis=1, inplace=True)

#### Filling dataframe with data from parameters

In [ ]:
df_obj["presentacion"] = df_obj.apply(lambda x: fill_meds_info(x), axis=1)
df_obj["presentacion"] = df_obj["presentacion"].str.capitalize()

df_obj["administracion"] = df_obj.apply(lambda x: fill_meds_adm(x), axis=1)

In [ ]:
df_obj

---

## Preparing information for export

In [ ]:
df_obj[df_obj.index==115]

In [ ]:
mr1 = df_obj["idm"] == 1
mr2 = df_obj["presentacion"] != "No_success"

df_res = df_obj.loc[mr1 & mr2, :].copy()

rc = ["medicamento", "presentacion", "administracion"]
df_res.drop([col for col in df_res.columns if col not in rc], axis=1, inplace=True)

df_res

In [ ]:
df_res.to_excel("../results/cat_meds_raw.xlsx")

In [ ]:
df_n = pd.read_csv("../utils/manual_notes.csv")
df_n

In [ ]:
x = df_n[df_n["b"].notnull()].loc[:, "a"].to_list()

In [ ]:
df_res2 = pd.read_excel("../results/cat_meds_notes.xlsx")

In [ ]:
df_res2 = df_res2[df_res2["Unnamed: 4"]!=1].copy()

df_res2.drop(["Unnamed: 0", "Unnamed: 4"], axis=1, inplace=True)

df_res2.reset_index(inplace=True, drop=True)

df_res2.to_excel("../results/cat_meds_rev.xlsx")

df_res2

---

## *Notes*

### Analyzing possible presentations

In [ ]:
pres_t = "BTE"

In [ ]:
dfx = df_obj.groupby(["pres_type", "pres_dims_gen"]).count()[["medicamento"]]

In [ ]:
dfx[dfx.index.get_level_values(0)==pres_t]

In [ ]:
mr1 = df_obj["pres_type"]==pres_t
mr2 = df_obj["pres_dims_gen"]=="#"

df_obj.loc[(mr1 & mr2), :]#.index.to_list()

In [ ]:
mr1 = df_obj["pres_type"]==pres_t
mr2 = df_obj["pres_dims_gen"]=="#"

df_obj.loc[(mr1 & ~mr2), :]#.index.to_list()

### Testing Levenshtein distance algorithm

In [ ]:
txt_a = "360 ANTISEPTICO JALOMA"

txt_match = [
    "360 NTISEPCO JALOA",
    ";akjdncn;a a;dkj ;akwejn;qkejn gfkdj slkerjbal;",
    "JALOMA ANTISEPTICO",
    "360 ANTISEPTICO A;AKJNR JALOMA AS;DKJRNLA"
]

In [ ]:
for txt_m in txt_match:
    print("{} --> {}".format(txt_m, rel_leveshtein(txt_a, txt_m)))

### Naive algorithm to tackle problem

### Test fuzzy matching

In [ ]:
df1 = pd.DataFrame({'Key':['Apple', 'Banana', 'Orange', 'Strawberry']})
df2 = pd.DataFrame({'Key':['Aple', 'Mango', 'Orag', 'Straw', 'Bannanna', 'Berry']})

In [ ]:
df1

In [ ]:
df2

In [ ]:
fuzzy_merge(df1, df2, 'Key', 'Key', threshold=80)

### Test difflib
- Notes:

In [ ]:
df1 = pd.DataFrame([[1],[2],[3],[4],[5], [11]], index=['one','two','three','four','five', "eleven"], columns=['number'])
df2 = pd.DataFrame([['a'],['b'],['c'],['d'],['e'], ["f"],["g"]], index=['onee','too','three','fours','fivvee', "six", "one"], columns=['letter'])

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1["match"] = df1.index.map(lambda x: difflib.get_close_matches(x, df2.index))

In [ ]:
df1

---
---